# Title
**Author**:  Greg Slater <br>
**Date**:  24th September 2024 <br>
**Dataset Scope**: `dataset` <br>
**Report Type**: Ad-hoc analysis <br>

## Purpose


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import urllib
# from sqlite_query_functions import DatasetSqlite
from datetime import datetime

pd.set_option("display.max_rows", 100)

td = datetime.today().strftime('%Y-%m-%d')
data_dir = "../../data/BFL_old_orgs/"
os.makedirs(data_dir, exist_ok=True)
# fn = os.path.join(data_dir, f"FILENAME_{td}.csv")


## Data Import

In [ ]:
bfl = pd.read_csv("https://files.planning.data.gov.uk/dataset/brownfield-land.csv")

In [ ]:
bfl_BBC = bfl[bfl["organisation-entity"] == 47].copy()
bfl_BCP = bfl[bfl["organisation-entity"] == 54].copy()

print(len(bfl_BCP))

In [ ]:
cross_ref = pd.merge(
    bfl_BBC,
    bfl_BCP,
    how = "outer",
    on = ["reference"],
    indicator=True,
)[["reference", "entity_x", "entity_y", "point_x", "point_y", "site-address_x", "site-address_y", "_merge"]]

matches = cross_ref[cross_ref["_merge"] == "both"].copy()
matches.sort_values(by = "reference", inplace=True)

matches.head()

In [ ]:
matches.iloc[1]["site-address_x"]

In [ ]:
matches.iloc[1]["site-address_y"]

In [ ]:
matches_BBC = bfl_BBC[bfl_BBC["reference"].isin(matches["reference"])].copy()
matches_BBC.set_index("reference", inplace=True)
matches_BBC.sort_index(inplace=True)

matches_BCP = bfl_BCP[bfl_BCP["reference"].isin(matches["reference"])].copy()
matches_BCP.set_index("reference", inplace=True)
matches_BCP.sort_index(inplace=True)

matches_BBC.head()

In [ ]:
full_comp = matches_BBC == matches_BCP
# full_comp.reset_index(inplace=True)
pd.DataFrame({"x_y_match_pct" : full_comp[["name", "point", "site", "site-address"]].sum(axis=0) / len(full_comp)})

## Analysis